In [44]:
from pyscf import gto, scf
import scipy
import inspect
import pyscf.qmmm
import pyscf.dft
import numpy as np
angstrom = 1 / 0.52917721067
from matplotlib import pyplot as plt
from functools import reduce
from numpy.linalg import inv 
from pyscf.scf import cphf 
# ancora il modo migliore per mettere cariche frazionali sulle molecole, se vai a toccare mol.charges si incazza 
#  quando deve calcolare l' initial guess 
def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords()/angstrom, deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [245]:
co=gto.Mole(atom="C 0 0 0 ;O 0 0 {}".format(2/angstrom), basis= "3-21g")
hfco=scf.RHF(co)
hfco.scf()
dL=.01
hf1=fc(hfco,[dL,-dL])
hf1.scf()
hf2=fc(hfco,[-dL,dL])
hf2.scf()
dV=DeltaV(co,[dL,-dL])
h2=hfco.get_veff()
dh2=(hf1.get_veff()-hf2.get_veff())/2
J,K=hfco.get_jk()
g_ijkl=hfco.mol.intor('int2e_sph')
g_h2=g_ijkl-np.swapaxes(g_ijkl,1,2)/2
nocc= co.nelec[0]

C=hfco.mo_coeff
S=hfco.get_ovlp()
e=hfco.mo_energy
E=np.diag(e)
O=np.diag(hfco.mo_occ)
C1=hf1.mo_coeff
P=hfco.make_rdm1()
P1=hf1.make_rdm1()
P2=hf2.make_rdm1() 
dP=(P1-P2)/2
dP2=(P1-2*P+P2)/dL**2
dC=(abs(C1)*C/abs(C)-C)

Initialize <pyscf.gto.mole.Mole object at 0x7f80e54d81d0> in <pyscf.scf.hf.RHF object at 0x7f80e54a3150>


converged SCF energy = -112.078731735567
converged SCF energy = -112.003586765744
converged SCF energy = -112.15425516577


In [246]:
G_mo=g_ijkl.copy()
G_mo=G_mo@C
G_mo=(G_mo.swapaxes(2,3)@C).swapaxes(3,2)
G_mo=(G_mo.swapaxes(1,3)@C).swapaxes(3,1)
G_mo=(G_mo.swapaxes(0,3)@C).swapaxes(3,0)

In [247]:
G_cphf=g_ijkl.copy()
G_cphf=4*G_cphf -G_cphf.swapaxes(3,1)-G_cphf.swapaxes(1,2)
G_cphf=G_cphf@C
G_cphf=(G_cphf.swapaxes(2,3)@C).swapaxes(3,2)
G_cphf=(G_cphf.swapaxes(1,3)@C).swapaxes(3,1)
G_cphf=(G_cphf.swapaxes(0,3)@C).swapaxes(3,0)
A=G_cphf.copy()
for i in range(co.nao):
    for a in range(co.nao):
        if e[i] !=e[a] :
            A[i,a,:,:]/=(e[a]-e[i])

In [248]:
dV_mo=C.T@dV@C
U_0= dV_mo.copy()
for i in range(co.nao):
    for j in range(co.nao):
        if e[i]!=e[j]:
            U_0[i][j]/=(e[j]-e[i])
Us=[]
Us.append(U_0)
for k in range(30):
    U=np.sum(u for u in Us)
    dP_app=C@(U@O-O@U)@C.T
    print(np.linalg.norm(dP_app-dP))
    Us.append(np.einsum('ijkl,kl->ij',A[:,:,nocc:,:nocc],Us[k][nocc:,:nocc])) # approximate value of U
    U=np.sum(u for u in Us)
    dP_app=C@(U@O-O@U)@C.T
    
print(np.linalg.norm(dP_app-dP))

0.011662644865576813
0.009309368387769615
0.00752775512272306
0.006123426158900554
0.004972592461064877
0.004041268530716813
0.003283659072246086
0.00266818390033354
0.002168135990979001
0.0017616987285573483
0.0014315639025042257
0.0011631820502966451
0.0009452275913460928
0.0007680006204149282
0.0006241152742649916
0.0005070748372643435
0.0004120950657715292
0.00033479383627614594
0.00027210488501997604
0.00022104218341870147
0.00017967384143492734
0.00014593560796427357
0.00011864459225001017
9.634516048626248e-05
7.834896381175076e-05
6.360221913517575e-05
5.1743108122284945e-05
4.198317800797395e-05
3.41762079148833e-05
2.7708936506546167e-05
2.2577493850241497e-05


/home/giorgiod/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  # Remove the CWD from sys.path while we load stuff.
/home/giorgiod/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  


In [249]:
co.energy_nuc()

24.000000011338358

In [250]:
dEnn=1/2.
print("true target energy: ",fc(hfco,[1,-1]).scf())
print("CO energy = ",hfco.e_tot)
print("APDFT1 energy =", hfco.e_tot+dEnn+np.trace(P.dot(dV/dL)))
print("APDFT2 energy =", hfco.e_tot+dEnn+np.trace((P+0.5*dP/dL).dot(dV/dL)))
print("APDFT2 energy approx.=", hfco.e_tot+dEnn+np.trace((P+ 0.5*dP_app/dL).dot(dV/dL)))
print("APDFT3 energy =", hfco.e_tot+dEnn+np.trace((P+0.5*dP/dL+(1/6)*dP2).dot(dV/dL)))
print("2nd deriv =", np.trace(dP.dot(dV/dL)/dL))
print("3rd deriv =", np.trace(dP2.dot(dV/dL)))

converged SCF energy = -106.480338236127
true target energy:  -106.48033823612661
CO energy =  -112.07873173556735
APDFT1 energy = -105.04530939648711
APDFT2 energy = -106.43761472926309
APDFT2 energy approx.= -106.43949971906133
APDFT3 energy = -106.45761188502304
2nd deriv = -2.784610665551977
3rd deriv = -0.11998293455964376


In [251]:
4*np.trace(U[:nocc,nocc:]@dV_mo[nocc:,:nocc])/dL**2

2.7883806451484627

$$ \frac {\partial^2 E}{\partial Z^2}= 4 \sum_a \sum_i U_{ia} \Delta V^{mo}_{ai}
$$

In [259]:
B_aa= np.einsum('ijkl,kl->ij',4*G_mo- G_mo.swapaxes(1,3)-G_mo.swapaxes(1,3).swapaxes(1,2),U[:,:nocc]@U.T[:nocc,:])/dL
print(-4*(B_aa[nocc:,:nocc]@U.T[:nocc,:nocc]).trace())
B_aa-=2*dV_mo@U /dL**2
print(-4*(B_aa[nocc:,:nocc]@U.T[:nocc,:nocc]).trace())
B_aa-=2*np.einsum('ijkl,kl->ij',(4*G_mo-G_mo.swapaxes(1,2)-G_mo.swapaxes(1,3))[:,:,:,:nocc],U[:,:nocc])@U
print(-4*(B_aa[nocc:,:nocc]@U.T[:nocc,:nocc]).trace())
B_aa+=0.5*np.einsum('ijkl,kl->ij',(4*G_mo -G_mo.swapaxes(1,3)-G_mo.swapaxes(1,2) )[:,:,:nocc,:],(U@U.T)[:nocc,:])/dL**2
print(-4*(B_aa[nocc:,:nocc]@U.T[:nocc,:nocc]).trace())
C=np.ones((e.size,e.size))
C[:,:]*=e
C.T[:,:]-=e
B_aa+=(U@U).T*C/dL**2
print(-4*(B_aa[nocc:,:nocc]@U.T[:nocc,:nocc]).trace())


-0.0023158023459398354
-0.003171393079384398
-0.0031687190522628594
-0.11875449349820695
-0.11945753780946976


In [262]:
-4*(B_aa[nocc:,:nocc]@U.T[:nocc,:nocc]).trace()/dL+4*(U.T[:nocc,nocc:]@dV_mo[nocc:,nocc:]@U[nocc:,:nocc]).trace()/dL**3

-11.886692111599727

In [254]:
4*(U.T[:nocc,nocc:]@dV_mo[nocc:,nocc:]@U[nocc:,:nocc]).trace()/dL**3

0.05906166934724887